In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Basic App: Question & Answering from Document

In [ ]:
from langchain_openai import OpenAI

In [ ]:
llm = OpenAI()

### Load text file

In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
loader = TextLoader("data/be-good.txt")

In [ ]:
document = loader.load()

### Document is loaded as Python list with metadata

In [ ]:
print(type(document))

In [ ]:
print(len(document))

In [ ]:
print(document[0].metadata)

In [ ]:
print(f"You have {len(document)} document.")

In [ ]:
print(f"Your document has {len(document[0].page_content)} characters")

### Split document in small chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [ ]:
document_chunks = text_splitter.split_documents(document)

In [ ]:
print(f"Now you have {len(document_chunks)} chunks.")

### Convert text chunks in numeric vectors (called "embeddings")

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()

### Load embeddings to vector database

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

### Create Retrieval Question & Answering Chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

### Now we have Question & Answering App

In [ ]:
question = """
What is this article about? 
Describe it in less than 100 words.
"""

In [ ]:
QA_chain.run(question)

In [ ]:
question2 = """
And how does it explain how to create somethin people want?
"""

In [ ]:
QA_chain.run(question2)

## Simple Agent

In [ ]:
#!pip install numexpr

In [ ]:
from langchain.agents import load_tools
from langchain.llms import OpenAI
from langchain.agents import AgentType

llm=OpenAI()

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

In [ ]:
agent.run("What is 133 by 142?")

### Let's make agent fail

In [ ]:
agent.run("Who was the wife of Napoleon Bonaparte?")

## Custom Agent

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun
from langchain.callbacks.manager import CallbackManagerForToolRun

In [ ]:
class CustomSearchTool(BaseTool):
    name = "article search"
    description = "useful for when you need to answer questions about our article"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = stored_embeddings.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    max_iterations=3
)

In [ ]:
agent.run("What is this article about? Describe it in less than 100 words.")